In [1]:
import os

In [2]:
os.chdir("../../")

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Laptop_datasplit:
    root_dir: Path
    laptop_cleaned_data: Path
    X_train: Path
    X_test: Path
    y_train: Path
    y_test: Path
    test_size: float
    random_state: int

In [5]:
from src.laptop.utils.common import read_yaml,create_directory
from src.laptop.constants import *

[2024-10-05 18:33:09,755: INFO]: Logging file start


In [6]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        create_directory([self.config.data_root])
    
    def get_laptop_data_split(self) -> Laptop_datasplit:
        config = self.config.data_split
        param = self.params.split
        create_directory([config.root_dir])

        model_datasplitConfig = Laptop_datasplit(
            root_dir = config.root_dir,
            laptop_cleaned_data = config.laptop_cleaned_data,
            X_train = config.X_train,
            X_test = config.X_test,
            y_train = config.y_train,
            y_test = config.y_test,
            test_size = param.test_size,
            random_state = param.random_state
        )

        return model_datasplitConfig



In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
class LaptopDatasplit:
    def __init__(self,config:Laptop_datasplit):
        self.config = config
    
    def load_data(self):
        self.df = pd.read_csv(self.config.laptop_cleaned_data)
        return self.df
    
    def Split_the_data(self):
        df = self.df
        X=df.drop('price',axis=1)
        y=df['price']
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=self.config.random_state,random_state=self.config.random_state)
        return X_train,X_test,y_train,y_test
    
    def remove_unwanted_cols_and_save(self):
        X_train,X_test,y_train,y_test = self.Split_the_data()
        X_train.drop('Unnamed: 0',inplace=True,axis=1)
        X_test.drop('Unnamed: 0',axis=1,inplace=True)
        X_train.to_csv(self.config.X_train)
        X_test.to_csv(self.config.X_test)
        y_train.to_csv(self.config.y_train)
        y_test.to_csv(self.config.y_test)
        print(X_train.head(2))
    

In [9]:
try: 
    config = ConfigurationManger()
    split_config = config.get_laptop_data_split()
    ld_split = LaptopDatasplit(config=split_config)
    ld_split.load_data()
    ld_split.Split_the_data()
    ld_split.remove_unwanted_cols_and_save()
except Exception as e:
    raise e

[2024-10-05 18:33:10,535: INFO]: yaml: config/laptop_config.yaml loaded successfully
[2024-10-05 18:33:10,535: INFO]: yaml: params/laptop_params.yaml loaded successfully
[2024-10-05 18:33:10,536: INFO]: created directory at: data
[2024-10-05 18:33:10,537: INFO]: created directory at: data/laptop/data_split
     Company  Product  TypeName    Inches  ScreenResolution  Cpu  Ram  Memory  \
471        7      594         3  0.408772                 8   74    8      26   
23         4      244         0 -1.204407                 7   83    8      16   

     Gpu  OpSys    Weight  
471    1      5 -0.208553  
23    61      5 -0.629467  
